#### Runnables in LangChain

#### What Are Runnables in LangChain?
- **Definition**: 
  A Runnable is an abstraction for components that perform a specific operation in a chain or pipeline. They are designed to work seamlessly with other components in LangChain.

- **Key Features**:
  1. **Composability**: 
     - Runnables can be combined (or chained) together to form larger workflows.
     - Example: The output of one Runnable can be the input for the next.

  2. **Execution via `invoke()`**:
     - Every Runnable can be executed using the `invoke()` method.
     - The method:
       - Takes an **input** (e.g., a string or a dictionary).
       - Processes it using the logic of the Runnable.
       - Returns an **output**.
---
#### Examples of Runnables:
1. **Prompt Templates**:
   - Used to format prompts dynamically.
   - Example: Filling in placeholders in a template.
     ```python
     prompt.invoke({"input": "Hello"})
     ```

2. **LLM Models**:
   - Represent language models like OpenAI or Hugging Face.
   - Example: Sending a query and receiving the model’s response.
     ```python
     llm.invoke("What is AI?")
     ```

3. **Output Parsers**:
   - Used to parse and structure the raw output of a model.
   - Example: Converting a plain text response into JSON.
     ```python
     parser.invoke("Name: John, Age: 30")
     ```

4. **Custom Functions**:
   - Any user-defined logic wrapped in a Runnable.
   - Example: Transforming or filtering input data.
     ```python
     custom_runnable.invoke(input_data)
     ```
---

#### LangChain Expression Language
**LangChain Expression Language** allows any two runnables to be “chained” together into sequences. The output of the previous runnable's .invoke() call is passed as input to the next runnable. This can be done using the pipe operator (|) or the more explicit .pipe() method, which performs the same function.

We shall learn about 4 types of Runnables  

**RunnablePassThrough:** Passes any input as it is to the next component in chain.</br>
**RunnableLambda:** Allows to convert any Python function into runnable object which can then be used in chain.</br>
**RunnableSequence:**  In a RunnableSequence, the output of each runnable serves as the input for the next runnable in the sequence.</br>
**RunnableParallel:** Passes input to parallel paths simultaneously.</br>


#### RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple examples: a chain with just RunnablePassthrough() will output the original input without any modification.

In [1]:
# Example-1
from langchain_core.runnables import RunnablePassthrough
chain = RunnablePassthrough()
chain = chain.invoke("DP Sharma")  # Here input given is DP Sharma and the RunnablePassthrough passon as it is. so output of the programm will be DP Sharma.
print(chain) # Wahtever input given same output
# here you can thinklike   RunnablePassthrough().invoke("DP Sharma") so input is "DP Sharma " and the output of the RunnablePassthrough is "DP Sharma". 

DP Sharma


In [2]:
# Example-2
from langchain_core.runnables import RunnablePassthrough

# Create a simple function to modify input
def modify_input(input_data):
    return f"Modified: {input_data}"

# Initialize RunnablePassThrough, i.e creating the object of RunnablePassThrough
pass_through = RunnablePassthrough()  # We can describe RunnablePassthrough as functioning like a placeholder for runtime input. It essentially allows you to pass the input data through unchanged

# Input data
input_data = "Hello, Langchain!"

# Modify the input
modified_output = modify_input(input_data)

# Pass the original input through
original_output = pass_through.invoke(input_data)

# Print both outputs
print("Modified Output:", modified_output)
print("Original Output:", original_output)

Modified Output: Modified: Hello, Langchain!
Original Output: Hello, Langchain!


#### RunnableLambda
* RunnableLambda can convert any normal Python function into a Runnable function that can be seamlessly used in LangChain workflows.
* Let's define a very simple function to create Russian lastnames:

In [3]:
# Example-3
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
def russian_lastname(name: str) -> str:
    return f"{name}ovich"
chain = RunnablePassthrough() | RunnableLambda(russian_lastname)
chain.invoke("DP Sharma")

'DP Sharmaovich'

#### RunnableSequence
* The RunnableSequence is used to chain multiple Runnables together, where the output of one Runnable becomes the input for the next.
* RunnableSequence approach is beneficial for creating structured, readable, and maintainable workflows in your applications, allowing complex operations to be broken down into simple, composable parts. 

##### Visual O/p of below Programme
Input: "Alice"  
      |  
      v  
greet_runnable → "Hello, Alice!"  
      |  
      v  
datetime_runnable → "Hello, Alice! The current date and time is YYYY-MM-DD."  
      |  
      v  
uppercase_runnable → "HELLO, ALICE! THE CURRENT DATE AND TIME IS YYYY-MM-DD."  
      |  
      v  
exclamation_runnable → "HELLO, ALICE! THE CURRENT DATE AND TIME IS YYYY-MM-DD.!"


In [4]:
# Example-4 : Runnable Lambda and Runnable Sequence
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnableSequence
# Define the transformations as simple functions
def greet(name):
   return f"Hello, {name}!"

def append_datetime(text):
   current_datetime = datetime.now().strftime("%Y-%m-%d")
   return f"{text} The current date and time is {current_datetime}."

def to_uppercase(text):
   return text.upper()

def add_exclamation(text):
   return f"{text}!"

# Wrap the functions in RunnableWrapper
greet_runnable = RunnableLambda(lambda x: greet(x))
datetime_runnable = RunnableLambda(lambda x: append_datetime(x))
uppercase_runnable = RunnableLambda(lambda x: to_uppercase(x))
exclamation_runnable = RunnableLambda(lambda x: add_exclamation(x))

##########################  # <<<<<< Either  >>>>>> #

# Create a RunnableSequence with the wrapped runnables
# chain = RunnableSequence(
#    first=greet_runnable,
#    middle=[datetime_runnable, uppercase_runnable],
#    last=exclamation_runnable,
# )

#########################   <<<<  OR    >>>>


# # Create a RunnableSequence with the wrapped runnables
# chain = RunnableSequence(
#  greet_runnable,
#  datetime_runnable,
#  uppercase_runnable,
#  exclamation_runnable,
# )

#########################   <<<<  OR    >>>>

chain = greet_runnable | datetime_runnable | uppercase_runnable | exclamation_runnable
print(chain)
# Apply the chain to some input data
input_data = "Alice"
result = chain.invoke(input_data)
print(result)

first=RunnableLambda(lambda x: greet(x)) middle=[RunnableLambda(lambda x: append_datetime(x)), RunnableLambda(lambda x: to_uppercase(x))] last=RunnableLambda(lambda x: add_exclamation(x))
HELLO, ALICE! THE CURRENT DATE AND TIME IS 2024-12-26.!


#### RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.
#### Important: the syntax of RunnableParallel can have several variations.

When composing a `RunnableParallel` with another `Runnable`, you do not need to wrap it up in the `RunnableParallel` class. Inside a chain, the next three syntaxes are equivalent:

1. `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
2. `RunnableParallel(context=retriever, question=RunnablePassthrough())`
3. `{"context": retriever, "question": RunnablePassthrough()}`

#### Basic of itemgetter 

```python
from operator import itemgetter
# Step 1: Prepare the "hidden function"
get_question = itemgetter("question")
# Step 2: Call the hidden function by passing the dictionary
result = get_question({"question": "What is Python?", "language": "English"})
print(result)  # Output: "What is Python?"

# Some more points about the `itemgetter`:

# 1. `Essentially setting up the itemgetter() object or function reference`
# 2. `Here we are invoking it by passing the dictionary`


In [5]:
from operator import itemgetter
# Step 1: Prepare the "hidden function"
get_question = itemgetter("question")
# Step 2: Call the hidden function by passing the dictionary
result = get_question({"question": "What is Python?", "language": "English"})
print(result)  # Output: "What is Python?"

What is Python?


In [6]:
# Example-5
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)
chain.invoke("Sharma")

{'operation_a': 'Sharma', 'operation_b': 'Sharmaovich'}

In [5]:
# Example-6
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define a parallel runnable , Here parallel input applied 
runnable = RunnableParallel(
    passed=RunnablePassthrough(),  # This will pass the input through unchanged
    modified=lambda x: x["num"] + 1,  # This will modify the input
)

# Invoke the runnable with a dictionary input
output = runnable.invoke({"num": 1})

# Print the output
print(output)


{'passed': {'num': 1}, 'modified': 2}


# Flow of Execution

1. **Import Required Libraries:**
   - Import `RunnableParallel` and `RunnablePassthrough` from `langchain_core.runnables`.

2. **Define a Parallel Runnable:**
   - Create an instance of `RunnableParallel` named `runnable`:
     - **`passed`**: Set to `RunnablePassthrough()`, which will pass the input through unchanged.
     - **`modified`**: A lambda function that takes an input dictionary and returns the value of `"num"` incremented by `1`.

3. **Invoke the Runnable:**
   - Call the `invoke` method on `runnable` with the input dictionary `{"num": 1}`.
   - This will pass the input through unchanged and also modify it by adding `1` to the value of `"num"`.

4. **Output:**
   - The `invoke` method returns a dictionary containing both:
     - The original input under the key `"passed"`.
     - The modified value under the key `"modified"`.
   - For the input `{"num": 1}`, the output will be:
     ```python
     {
         "passed": {"num": 1},
         "modified": 2
     }
     ```

5. **Print the Output:**
   - Print the output to the console to see the results.
